In [ ]:
from math import pi

import matplotlib.pyplot as plt

from ct import load_bitmap

# filename = "Kropka.jpg"
# filename = "Shepp_logan.jpg"
filename = "SADDLE_PE-large.JPG"

scans_count = 360

alfa_step = pi / scans_count

detectors_count = 360

detectors_angular_aperture = 0.9 * pi

has_to_use_sinogram_filter = True
has_to_save_intermediate_steps = False
has_to_show_intermediate_steps = False

image = load_bitmap(filename)
plt.imshow(image)
image.close()

In [ ]:
from ct import simulateComputedTomographyScan


states = simulateComputedTomographyScan(
    filename,
    scans_count,
    alfa_step,
    detectors_count,
    detectors_angular_aperture,
    has_to_use_sinogram_filter,
    has_to_save_intermediate_steps,
    has_to_show_intermediate_steps,
)

In [ ]:
import ipywidgets

from ct import showStateAfterScan


ipywidgets.interact(
    showStateAfterScan,
    states=ipywidgets.fixed(states),
    scan=ipywidgets.IntSlider(
        min=1 if states[-1] != 1 else scans_count, max=scans_count
    ),
)

In [ ]:
import pydicom

from ct import create_dicom_from_image

image_data = states[3]
dicom_filename = filename.split(".")[0] + ".dcm"
create_dicom_from_image(image_data, dicom_filename, "Example: image description", "Patient is healthy")

ds = pydicom.dcmread(dicom_filename)
print(ds)

TODO

https://docs.google.com/presentation/d/1SeYwXUH1SB114OJO8NVovHBeupTIpM4muwNpd7r4pO0/edit#slide=id.gc39bab9b14_1_224


2.
Należy dokonać prostej analizy statystycznej w oparciu o jedną zdefiniowaną miarę jakości. Np. mając obraz wejściowy i wyjściowy można policzyć błąd średniokwadratowy (po wszystkich pikselach różnicy obrazu wejściowego i wyjściowego). Taka analiza powinna uwzględnić następujące elementy:
zmianę błędu średniokwadratowego podczas wykonywania kolejnych iteracji odwrotnej transformaty Radona dla wybranego obrazu,
zmianę błędu średniokwadratowego przy zwiększaniu dokładności próbkowania (trzy uprzednio wymienione parametry modelu emiter/detektor), 
zmianę błędu średniokwadratowy po włączeniu filtrowania. 

3.
Raport
https://docs.google.com/document/d/15zb5myHaLJqZUuv3Wv-gWXCP6wZn44s8QhGeW3izCGg/edit?tab=t.0
